In [6]:
# Import required libraries
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    confusion_matrix, classification_report, ConfusionMatrixDisplay,
    precision_score, recall_score, f1_score, roc_auc_score, 
    average_precision_score, accuracy_score, roc_curve, precision_recall_curve
)


# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

Libraries imported successfully!


In [7]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score, 
    average_precision_score, confusion_matrix, classification_report
)

In [8]:
RANDOM_STATE = 44
CLASS_WEIGHT = 'balanced'

In [9]:
# Ensure the notebook runs from project root so relative paths like 'data/creditcard.csv' work
import os
from pathlib import Path
print("Working directory before change:", os.getcwd())
project_root = Path('/Users/nolanrink/Desktop/School/Supervised-Learning-Final-Project')
if project_root.exists():
    os.chdir(project_root)
    print("Changed working directory to:", os.getcwd())
else:
    raise FileNotFoundError(f"Project root not found: {project_root}")
# Show top-level contents to confirm working directory
print('Top-level files/folders:', os.listdir('.'))

Working directory before change: /Users/nolanrink/Desktop/School/Supervised-Learning-Final-Project
Changed working directory to: /Users/nolanrink/Desktop/School/Supervised-Learning-Final-Project
Top-level files/folders: ['data', 'notebooks']


In [10]:
smote_x_train = pd.read_csv('data/processed/x_train_smote.csv')
smote_y_train = pd.read_csv('data/processed/y_train_smote.csv')

x_train = pd.read_csv('data/processed/x_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')

x_test = pd.read_csv('data/processed/x_test.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

In [11]:
def evaluate_model(y_true, y_pred, y_pred_proba, model_name="Model"):
    """
    Comprehensive model evaluation.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (for AUC)
        model_name: Name of the model (for printing)
        
    Returns:
        dict: Dictionary of metrics
    """
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
        'pr_auc': average_precision_score(y_true, y_pred_proba),
        'confusion_matrix': confusion_matrix(y_true, y_pred),
        'classification_report': classification_report(y_true, y_pred, zero_division=0)
    }
    
    # Print results
    print(f"\n{'='*70}")
    print(f"{model_name} - Test Set Results")
    print(f"{'='*70}")
    print(f"Accuracy:  {metrics['accuracy']:.4f} (% of correct predictions)")
    print(f"Precision: {metrics['precision']:.4f} (% of predicted fraud that are correct)")
    print(f"Recall:    {metrics['recall']:.4f} (% of actual fraud detected)")
    print(f"F1-Score:  {metrics['f1']:.4f} (harmonic mean of precision & recall)")
    print(f"ROC-AUC:   {metrics['roc_auc']:.4f} (area under ROC curve)")
    print(f"PR-AUC:    {metrics['pr_auc']:.4f} (area under precision-recall curve)")
    print(f"\nConfusion Matrix:")
    print(metrics['confusion_matrix'])
    print(f"\nClassification Report:")
    print(metrics['classification_report'])
    
    return metrics

In [12]:
"""
Data preprocessing pipelines: scaling, resampling, etc.
"""
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline


def get_scaler():
    """
    Get a StandardScaler for feature scaling.
    
    Returns:
        StandardScaler: Configured scaler
    """
    return StandardScaler()


def create_column_transformer_preprocessor():
    """
    Create a ColumnTransformer that:
    - Scales ['Time', 'Amount'] using StandardScaler
    - Passes V1-V28 (PCA features) through as-is
    
    Returns:
        ColumnTransformer: Configured transformer
    """
    # Identify PCA features (V1-V28)
    pca_features = [f'V{i}' for i in range(1, 29)]
    scale_features = ['Time', 'Amount']
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('scale', StandardScaler(), scale_features),
            ('passthrough', 'passthrough', pca_features)
        ],
        remainder='drop'
    )
    
    return preprocessor


def create_preprocessing_pipeline():
    """
    Create a complete preprocessing pipeline with ColumnTransformer.
    
    Returns:
        Pipeline: Sklearn pipeline with preprocessing
    """
    preprocessor = create_column_transformer_preprocessor()
    
    return Pipeline([
        ('preprocess', preprocessor)
    ])


def create_model_pipeline(model, with_preprocessing=True):
    """
    Create a full pipeline with preprocessing and model.
    
    Args:
        model: The classifier to use
        with_preprocessing (bool): Whether to include preprocessing (default True)
        
    Returns:
        Pipeline: Complete pipeline
    """
    if with_preprocessing:
        return Pipeline([
            ('preprocess', create_column_transformer_preprocessor()),
            ('clf', model)
        ])
    else:
        return Pipeline([
            ('clf', model)
        ])


def create_scaling_pipeline():
    """
    Create a simple pipeline for scaling all features.
    
    Returns:
        Pipeline: Sklearn pipeline with scaler
    """
    return Pipeline([
        ('scaler', StandardScaler())
    ])


def create_smote_pipeline():
    """
    Create a pipeline with SMOTE resampling and scaling.
    
    Returns:
        ImbPipeline: Imbalanced-learn pipeline with SMOTE
    """
    return ImbPipeline([
        ('smote', SMOTE(random_state=42)),
        ('scaler', StandardScaler())
    ])


def create_combined_resample_pipeline():
    """
    Create a pipeline combining oversampling (SMOTE) and undersampling.
    
    Returns:
        ImbPipeline: Imbalanced-learn pipeline with combined resampling
    """
    return ImbPipeline([
        ('smote', SMOTE(random_state=42)),
        ('undersample', RandomUnderSampler(random_state=42)),
        ('scaler', StandardScaler())
    ])


In [13]:
lr = LogisticRegression(
        class_weight=CLASS_WEIGHT,
        random_state=RANDOM_STATE,
        max_iter=10000,
        solver='lbfgs')

lr_smote = LogisticRegression(
        class_weight=CLASS_WEIGHT,
        random_state=RANDOM_STATE,
        max_iter=10000,
        solver='lbfgs')

In [14]:
# Create and train Logistic Regression
pipe_lr = create_model_pipeline(lr, with_preprocessing=False)

print("Training Logistic Regression...")
pipe_lr.fit(x_train, y_train)
print("✓ Training complete!")

# Make predictions
y_pred_lr = pipe_lr.predict(x_test)
y_pred_proba_lr = pipe_lr.predict_proba(x_test)[:, 1]

# Evaluate
metrics_lr = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")

Training Logistic Regression...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✓ Training complete!

Logistic Regression - Test Set Results
Accuracy:  0.9785 (% of correct predictions)
Precision: 0.0668 (% of predicted fraud that are correct)
Recall:    0.8878 (% of actual fraud detected)
F1-Score:  0.1243 (harmonic mean of precision & recall)
ROC-AUC:   0.9704 (area under ROC curve)
PR-AUC:    0.7601 (area under precision-recall curve)

Confusion Matrix:
[[55649  1215]
 [   11    87]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.07      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.56     56962
weighted avg       1.00      0.98      0.99     56962



In [20]:
# Create and train Logistic Regression
pipe_lr_preprocessed = create_model_pipeline(lr, with_preprocessing=True)

print("Training Logistic Regression...")
pipe_lr_preprocessed.fit(x_train, y_train)
print("✓ Training complete!")

# Make predictions
y_pred_lr = pipe_lr_preprocessed.predict(x_test)
y_pred_proba_lr = pipe_lr_preprocessed.predict_proba(x_test)[:, 1]

# Evaluate
metrics_lr = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")

Training Logistic Regression...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✓ Training complete!

Logistic Regression - Test Set Results
Accuracy:  0.9784 (% of correct predictions)
Precision: 0.0666 (% of predicted fraud that are correct)
Recall:    0.8878 (% of actual fraud detected)
F1-Score:  0.1238 (harmonic mean of precision & recall)
ROC-AUC:   0.9709 (area under ROC curve)
PR-AUC:    0.7616 (area under precision-recall curve)

Confusion Matrix:
[[55644  1220]
 [   11    87]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.07      0.89      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.93      0.56     56962
weighted avg       1.00      0.98      0.99     56962



In [15]:
# Create and train Logistic Regression
smote_pipe_lr = create_model_pipeline(lr_smote, with_preprocessing=True)

print("Training Logistic Regression...")
smote_pipe_lr.fit(smote_x_train, smote_y_train)
print("✓ Training complete!")

# Make predictions
y_pred_lr = smote_pipe_lr.predict(x_test)
y_pred_proba_lr = smote_pipe_lr.predict_proba(x_test)[:, 1]

# Evaluate
metrics_lr = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")

Training Logistic Regression...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✓ Training complete!

Logistic Regression - Test Set Results
Accuracy:  0.9917 (% of correct predictions)
Precision: 0.1557 (% of predicted fraud that are correct)
Recall:    0.8673 (% of actual fraud detected)
F1-Score:  0.2640 (harmonic mean of precision & recall)
ROC-AUC:   0.9741 (area under ROC curve)
PR-AUC:    0.7547 (area under precision-recall curve)

Confusion Matrix:
[[56403   461]
 [   13    85]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.16      0.87      0.26        98

    accuracy                           0.99     56962
   macro avg       0.58      0.93      0.63     56962
weighted avg       1.00      0.99      0.99     56962



In [16]:
lr = LogisticRegression(
        class_weight=CLASS_WEIGHT,
        random_state=RANDOM_STATE,
        max_iter=10000,
        solver='lbfgs')

lr_smote = LogisticRegression(
        class_weight=CLASS_WEIGHT,
        random_state=RANDOM_STATE,
        max_iter=10000,
        solver='lbfgs')

In [17]:
def compute_metrics(y_true, y_pred, y_pred_proba=None):
    """
    Compute multiple evaluation metrics.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        y_pred_proba: Predicted probabilities (for AUC scores)
        
    Returns:
        dict: Dictionary of metrics
    """
    metrics = {
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'confusion_matrix': confusion_matrix(y_true, y_pred)
    }
    
    if y_pred_proba is not None:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba)
        metrics['pr_auc'] = average_precision_score(y_true, y_pred_proba)
    
    return metrics


In [18]:
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
metrics = compute_metrics(y_test, y_pred, lr.predict_proba(x_test)[:, 1])
for metric, value in metrics.items():
    print(f"{metric}: {value}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


precision: 0.06682027649769585
recall: 0.8877551020408163
f1: 0.12428571428571429
confusion_matrix: [[55649  1215]
 [   11    87]]
roc_auc: 0.9703988499592296
pr_auc: 0.760082195166476


In [19]:
lr_smote.fit(smote_x_train, smote_y_train)
y_pred = lr_smote.predict(x_test)
metrics = compute_metrics(y_test, y_pred, lr_smote.predict_proba(x_test)[:, 1])
for metric, value in metrics.items():
    print(f"{metric}: {value}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


precision: 0.15769944341372913
recall: 0.8673469387755102
f1: 0.2668759811616955
confusion_matrix: [[56410   454]
 [   13    85]]
roc_auc: 0.974548654577194
pr_auc: 0.755887640870852
